# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - BIKE

Implementar o esquema BIKE em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento BIKE (https://bikesuite.org/files/BIKE.pdf)

### BIKE - KEM-IND-CPA

Para o desenvolvimento deste KEM de forma a ser IND-CPA, aquilo que fizemos foi seguir então o documento especificado, mais precisamente o algoritmo denominado **BIKE-1**. 

Antes de mais nada, importante salientar que este algoritmo utiliza os parâmetros de segurança $N$, $R$, $W$ e $T$, que são passados como parâmetro. Além disso, na inicialização da classe são criados o $K2$ (corpo finito de tamanho 2) e o $R$ (anél quociente de polinómios $F[X]/<X^r + 1>$).

De seguida, tentaremos explicar detalhadamente as funções responsáveis por gerar as chaves pública e privada (*geraChaves()*), a função responsável por gerar a chave k e o seu encapsulamento (*encapsula()*) e a função responsável por desencapsular a chave a partir do seu 'encapsulamento' (*desencapsula()*).

**Geração da chave pública *(f0,f1)* e da chave privada *(h0,h1)* *(função geraChaves())***

  - Primeiro, gera-se os parâmetros $h0$ e $h1$ pertencentes a $R$ ambos com peso de hamming w/2, ou seja, o número de coeficientes do polinómio iguais a 1 tem de ser w/2;
  - Depois foi necessário gerar um polinómio $g$ pertencente a $R$ com peso de hamming r/2;
  - Por fim, é só computar (f0, f1) $<-$ (gh1, gh0) e retornar a chave privada (h0,h1) e a chave pública (f0,f1).


**Encapsulamento e geração da chave *(função encapsula())***

  - A função encapsula recebe como parâmetro a chave pública *(f0,f1)* e retorna o para (k, c), onde c é o 'encapsulamento' da chave e o k é a chave em si. De notar que esta função foi dividida propositadamente para ao se fazer o **PKE-IND-CCA** se poder aplicar facilmente a transformação de Fujisaki-Okamoto;
  - Sendo assim, na função *h()*, aquilo que é feito é a geração de dois erros $e0$ e $e1$ pertencentes a $R$ de tal forma que o peso de hamming de $e0$ mais o peso de hamming de $e1$ tem de ser igual a $t$ (|e0| + |e1| = t). Além disso, esta função gera também um $m$ pertencente a  $R$ de forma aleatória e que deve ser denso;
  - Por último, a função *f()*, que já é mais determinista, recebe como parâmetros a chave pública, o parâmetro m e os erros (e0,e1) e retorna então o par (k,c). Muito resumidamente, os cálculos que são efetuados são os seguintes:
    - Cálculo de c = (c0, c1) $<-$ (m.f0 + e0, m.f1 + e1), em que c é o encapsulamento da chave;
    - Cálculo de k $<-$ Hash(e0, e1), em que k é a chave simétrica.


**Desencapsulamento da chave *(função desencapsula())***

  - A função desencapsula recebe como parâmetros a chave privada *(h0,h1)* e ainda o 'encapsulamento' da chave $c$ e retorna a chave $k$. Esta função, tal como a função *encapsula()*, também teve de ser separada em duas funções: *find_errorVec()* (serve para descodificar os vetores de erro $e0$ e $e1$) e a função *calculateKey()* (é a que vai calcular efetivamente a chave);
  - Na função *find_errorVec()*, começamos por converter o encapsulamento da chave num vetor em n, sendo este o código usado aquando do *bitFlip()*;
  - Depois, formamos a matriz H = (rot(h0)|rot(h1));
  - Cálculo do síndrome s $<-$ c0.h0 + c1.h1, ou seja faz-se a multiplicação do código com a matriz H;
  - Depois, tenta-se descodificar $s$ usando o algoritmo *bitFlip()* para recuperar o vetor (e0, e1). Isso é feito fazendo o *bitFlip()* e com esse resultado efetuar uma série de cálculos;
  - Depois de obtido o resultado do *bitFlip()*, colocámos esse resultado em forma de par de polinómios (bf0,bf1);
  - Por fim, e como estamos a falar de um código sistemático, o m = bf0 e o (e0,e1) é cálculado como:
    - e0 = c0 - bf0 * 1;
    - e1 = c1 - bf0 * sk0/sk1.



In [2]:
import random, hashlib

# Baseado no esquema da página BIKE-1 do documento https://bikesuite.org/files/BIKE.pdf

class BIKE_KEM(object):
    
    def __init__(self, N, R, W, T, timeout=None):
        
        # Numero primo r que e passado como argumento
        self.r = R
        # Normalmente, n = 2*r
        self.n = N
        self.w = W
        # t é um número inteiro usado na descodificação
        self.t = T
        
        # Corpo finito de tamanho 2
        self.K2 = GF(2)
        # Polynomial Ring in x over Finite Field of size 2
        F.<x> = PolynomialRing(self.K2)
        # The cyclic polynomial ring F[X]/<X^r + 1>
        R.<x> = QuotientRing(F, F.ideal(x^self.r + 1))
        self.R = R
    
    
    # Calcula a Hamming weight de um vetor(numero de não zeros em representacao binaria)
    def hammingWeight(self, x):
        
        return sum([1 if a == self.K2(1) else 0 for a in x])
        
        
    # Gera aleatoriamente os coeficientes binarios de um polinomio com w 1's e de tamanho n
    def gera_Coef(self, w, n):
        
        res = [1]*w + [0]*(n-w-2)
        random.shuffle(res)
        return self.R([1]+res+[1])
    
    
    # Gera um par de polinomios de tamanho "r" com um número total de erros (1's) "w"
    def gera_CoefP(self, w):
        
        res = [1]*w + [0]*(self.n-w)
        random.shuffle(res)
        return (self.R(res[:self.r]), self.R(res[self.r:]))
    
    
    # Converte uma lista de coeficientes em dois polinómios
    def convert_Pol(self, e):
        
        u = e.list()
        return (self.R(u[:self.r]), self.R(u[self.r:]))

    
    #função para calcular o hash
    def Hash(self, e0, e1):
        
        m = hashlib.sha3_256()
        m.update(e0.encode())
        m.update(e1.encode())
        return m.digest()
    
    
    # componentwise product of vectors
    def componentwise(self, v1, v2):
        
        return v1.pairwise_product(v2)
    
    
    # converter para vetor um polinomio de tamanho r
    def vectorConverter_r(self, p):
        
        V = VectorSpace(self.K2, self.r)
        return V(p.list() + [0]*(self.r - len(p.list())))
    
    
    # converter para vetor um tuplo de polinomios de tamanho n
    def vectorConverter_n(self, pp):
        
        V = VectorSpace(self.K2, self.n)
        f = self.vectorConverter_r(pp[0]).list() + self.vectorConverter_r(pp[1]).list()
        return V(f)
    
    
    # Roda uma unidade a mais os elementos de uma vetor
    def rot_vec(self, h):
        
        V = VectorSpace(self.K2, self.r)
        v = V()
        v[0] = h[-1]
        for i in range(self.r-1):
            v[i+1] = h[i]
            
        return v

    
    # Funcao que gera a matriz de rotacao a partir de um vetor
    def rot(self, v):
        
        # Cria uma matriz de 0 e 1 de tamanho (r x r)
        M = Matrix(self.K2, self.r, self.r)
        # transforma para vetor o v
        M[0] = self.vectorConverter_r(v)
        # Aplicar sucessivamente as rotacoes a todas as linhas da matriz
        for i in range(1, self.r):
            M[i] = self.rot_vec(M[i-1])
        return M
    
    
    # BitFlip
    # Recebe como parametro a matriz H = H0 + H1, o palavra de codigo y, o sindroma s
    # e ainda o numero de iteracoes maximas para descobrir tirar os erros(por questoes de eficiencia)
    def bitFlip(self, H, y, s, n_iter):
        
        # Nova palavra de codigo
        x = y
        # Novo sindroma
        z = s
        
        while self.hammingWeight(z) > 0 and n_iter > 0:
            
            # Gerar um vetor com todas os pesos de hamming de |z . Hi|
            pesosHam = [self.hammingWeight(self.componentwise(z, H[i])) for i in range(self.n)]
            maxP = max(pesosHam)
            
            for i in range(self.n):
                # Verificar de |hj . z| é grande
                if pesosHam[i] == maxP:
                    # Efetua o flip do bit
                    x[i] += self.K2(1)
                    # atualiza o sindroma
                    z += H[i]
            # Decresce o numero de iteracoes
            n_iter = n_iter - 1
        
        # Verificar se o numero de iteracoes foi atingido, caso tenha sido atingido lanca exceção
        if n_iter == 0:
            raise ValueError("o limite de iteracoes foi atingido!!!!")
        
        return x
    
    
    # Funcao que gera um polinomio aleatorio m <- R, denso e ainda um tuplo (e0,e1)
    # (Feito unicamente para aplicar F.O. no PKE-IND-CCA)
    def h(self):
        
        # Sample (e0,e1) € R such that |e0| + |e1| = t.
        e = self.gera_CoefP(self.t)
        # Gerar um m <- R, denso
        m = self.R.random_element()
        
        return (m,e)
    
    
    # Funcao que gera a chave e o seu 'encapsulamento' recebendo como parametro a chave publica,
    # um polinomio m que sera usado para gerar a chave simetrica e ainda um (e0,e1) € R 
    # (Feito unicamente para aplicar F.O. no PKE-IND-CCA)
    def f(self, pk, m, e):
        
        # Compute c = (c0, c1) <- (m.f0 + e0, m.f1 + e1).
        c0 = m * pk[0] + e[0]
        c1 = m * pk[1] + e[1]
        c = (c0,c1)
        # Compute K <- K(e0, e1), em que K e o Hash
        k = self.Hash(str(e[0]), str(e[1]))
        
        return (k, c)
    
    
    # Funcao que serve para descobrir com auxilio do bitflip o vetor de erro e (Feito unicamente para aplicar F.O. no PKE-IND-CCA)
    def find_errorVec(self, sk, c):
        
        # Converter o criptograma num vetor em n
        code = self.vectorConverter_n(c)
        # Formar a matriz H = (rot(h0)|rot(h1))
        H = block_matrix(2, 1, [self.rot(sk[0]), self.rot(sk[1])])
        # Compute the syndrome s <- c0.h0 + c1.h1.
        s = code * H
        # Try to decode s (noiseless) to recover an error vector (e0, e1).
        bf = self.bitFlip(H, code, s, self.r)
        # Colocar o e como um par de polinomios 
        (bf0, bf1) = self.convert_Pol(bf)
        # visto ser um código sistemático, m = bf0
        e0 = c[0] - bf0 * 1
        e1 = c[1] - bf0 * sk[0]/sk[1]
        
        return (e0,e1)
    
    
    # Funcao recebe o vetor de erro e retorna o calculo da chave (Feito unicamente para aplicar F.O. no PKE-IND-CCA)
    def calculateKey(self, e0, e1):
        
        # If |(e0,e1)| != t or decoding fails, output error and halt.
        if self.hammingWeight(self.vectorConverter_r(e0)) + self.hammingWeight(self.vectorConverter_r(e1)) != self.t:
            raise ValueError("Erro no decoding!!!")
        # Compute K <- K(e0, e1), em que K e o Hash
        k = self.Hash(str(e0), str(e1))
        
        return k
    
    
    # Função responsavel por gerar o par de chaves privada e publica
    def gera_Chaves(self):
        
        # Generate h0,h1 <- R both of (odd) weight |h0| = |h1| = w/2.
        h0 = self.gera_Coef(self.w//2, self.r)
        h1 = self.gera_Coef(self.w//2, self.r)
        # Generate g <- R of odd weight (so |g| = r/2).
        g = self.gera_Coef(self.r//2, self.r)
        # Compute (f0, f1) <- (gh1, gh0).
        f0 = g*h1
        f1 = g*h0
        
        return {'sk' : (h0,h1) , 'pk' : (f0, f1)}
    
    
    # Retorna a chave encapsulada K and the criptograma ("encapsulamento") c.
    def encapsula(self, pk):
        
        # Gerar um m <- R, denso
        (m,e) = self.h()
        
        return self.f(pk, m, e)
    
    
    # Retorna a chave desencapsulada k ou erro
    def desencapsula(self, sk, c):

        # Descodificar o vetor de erro 
        (e0, e1) = self.find_errorVec(sk, c)
        # Calcular a chave 
        k = self.calculateKey(e0, e1)
        
        return k
        

#### Testagem da classe definida acima:

In [3]:
# Parametros escolhidos unicamente para este cenario de teste
R = next_prime(1000)
N = 2*R
W = 6
T = 32

kem = BIKE_KEM(N,R,W,T)

# Gerar as chaves publica e privada
chaves = kem.gera_Chaves()
# Gerar uma chave e o seu encapsulamento
(k,c) = kem.encapsula(chaves['pk'])
# Desencapsular o 'encapsulamento' da chave para obter a chave
k1 = kem.desencapsula(chaves['sk'], c)

if k == k1:
    print("A chave desencapsulada é igual à original!!!")

A chave desencapsulada é igual à original!!!


### BIKE - PKE-IND-CCA

De acordo com o documento da submissão do **BIKE**, devido ao facto de no bit-flip poderem acontecer alguns erros, não é muito fácil atingir a segurança CCA. Porém, e depois de alguma pesquisa, concluíu-se que se usasse-mos a transformação de Fujisaki-Okamoto, era possível atingir algo perto desta noção.


**Geração das chaves pública *(f0,f1)* e privada *(h0,h1)* (feito na inicialização da classe)**

Para gerar a chave pública e privada para este esquema criptográfico, bastou-nos usar a geração de chave que acontece no **BIKE-KEM** definido acima. Assim, na inicialização da classe **BIKE_PKE** já é feito a inicialização da classe **BIKE_KEM** e consequentemente já se obtêm as chaves pública e privada a partir daí.


**Cifragem(função *cifra()*)**

A função de cifragem recebe como input a mensagem a cifrar e a chave pública $(f0,f1)$ e efetua os seguintes passos:

- Primeiramente, este algoritmo gera um polinomio aleatorio r <- R, denso e um par (e0,e1);

- Calculo do g(r), em que g é uma função de hash (no nosso caso, sha3-256);

- A seguir é feito o **XOR** entre a mensagem original e o hash do $r$ ($g(r)$), que deve ser do mesmo tamanho do que a mensagem original: y <- m $\oplus$ g(r);

- Depois, transforma-mos esta string de bytes numa string binária para depois converter esta string binária num polinómio em $R$;

- Depois é usada a função *f()* do **BIKE-KEM** para encapsular uma chave que é gerada a partir do parâmetro $r$ e do $y$ e obtem-se o par $(k,w)$ que representam respetivamente a chave e o 'encapsulamento' da chave;

- Por fim ofusca-se a chave através do **XOR** entre o $r$ e o $k$: c <- r $\oplus$ k. Assim, o resultado final desta função é o triplo constituído por $(y,w,c)$.


**Decifragem (função *decifra()*)**

A função de decifragem recebe como input a ofuscação da mensagem original $y$, o encapsulamento da chave $w$ e a ofuscação da chave $c$ e realiza as operações seguintes:

 - Desencapsula a chave através do encapsulamento da chave $w$ e da chave privada, sendo que esse desencapsulamento produz a chave $k$. O desencapsulamento é efetuado em duas fases através das funções *find_errorVec()* que nos devolve os erros (e0,e1) e na função *calculateKey()* que nos devolve efetivamente a chave;
 
 - Calcula o resultado de r <- c $\oplus$ k;
 
 - Depois, transforma-mos a string de bytes y numa string binária para depois converter esta string binária num polinómio em $R$;
 
 - Teste se o resultado do encapsulamento da chave é igual a $(w,k)$, através dos parâmetros $r$ e $y$. Caso o resultado do encapsulamento da chave não for igual a $(w,k)$, então dá erro e lança exceção. Caso contrário é então calculado o resultado de m <- y $\oplus$ g(r), dando portanto a mensagem original como resultado.


In [4]:
# Classe que implementa um PKE_IND_CCA (Public Key Encryption) a partir do BIKE_KEM e da transformação de Fujisaki-Okamoto
# Baseamo-nos em: https://bikesuite.org/files/BIKE.pdf e nos apontamentos das aulas teóricas


class BIKE_PKE(object):
    
    def __init__(self, N, R, W, T, timeout=None):
        
        # Inicializacao da classe KEM do BIKE
        self.kem = BIKE_KEM(N,R,W,T)
        # Geracao das chaves publica e privada
        self.chaves = self.kem.gera_Chaves()
    
    
    # XOR de 2 arrays de bytes byte-a-byte! A mensagem(data) deve ser menor ou igual á chave(mask)! Caso contrario, a chave
    # ou mask é 'repetida' para os bytes seguintes dos dados
    def xor(self, data, mask):
        
        masked = b''
        ldata = len(data)
        lmask = len(mask)
        i = 0
        while i < ldata:
            for j in range(lmask):
                if i < ldata:
                    masked += (data[i] ^^ mask[j]).to_bytes(1, byteorder='big')
                    i += 1
                else:
                    break
                    
        return masked
    
    
    # Funcao usada para cifrar que recebe a mensagem e uma chave publica
    def cifra(self, m, pk):
        
        # Gerar um polinomio aleatorio r <- R, denso e um par (e0,e1)
        (r,e) = self.kem.h()
        # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha3-256)
        g = hashlib.sha3_256(str(r).encode()).digest()
        # Efetuar o calculo de: y ← x⊕ g(r)
        y = self.xor(m.encode(), g)
        # Transformar esta string de bytes numa string binaria
        im = bin(int.from_bytes(y, byteorder=sys.byteorder))
        yi = self.kem.R(im)
        # Calcular (k,w) ← f(y || r)
        (k,w) = self.kem.f(pk, yi + r, e)
        # Calcular c ← k⊕ r
        c = self.xor(str(r).encode(), k)
        
        return (y,w,c)
    
    
    # Funcao usada para decifrar que recebe a chave privada, o criptograma, o 'encapsulamento' da chave 
    # e a 'ofuscação' da chave
    def decifra(self, sk, y, w, c):
        
        # Fazer o desencapsulamento da chave
        #k = self.kem.desencapsula(sk, w)
        e = self.kem.find_errorVec(sk, w)
        k = self.kem.calculateKey(e[0], e[1])
        # Calcula r <- c  ⊕  k
        rs = self.xor(c, k)
        r = self.kem.R(rs.decode())
        
        im = bin(int.from_bytes(y, byteorder=sys.byteorder))
        yi = self.kem.R(im)
        
        # Verificar se (w,k) ≠ f(y∥r)
        if (k,w) != self.kem.f(self.chaves['pk'], yi + r, e):
            # Lancar excecao
            raise IOError
        else:
            # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha-256)
            g = hashlib.sha3_256(rs).digest()
            # Calcular m <- y⊕ g(r)
            m = self.xor(y, g)
        
        return m
    

#### Testagem da classe definida acima:

In [5]:
# Parametros escolhidos unicamente para este cenario de teste
R = next_prime(1000)
N = 2*R
W = 6
T = 32

kem = BIKE_PKE(N,R,W,T)

m = input("Insira uma mensagem a cifrar: ")

(y,w,c) = kem.cifra(m, kem.chaves['pk'])

m1 = kem.decifra(kem.chaves['sk'], y, w, c)

if m == m1.decode():
    print("Decifragem ocorreu com sucesso!!!!")
    print("Mensagem decifrada foi: " + m1.decode())
else:
    print("Decifragem ocorreu sem sucesso!!!!")

Insira uma mensagem a cifrar: Viva a Estruturas Criptográficas :)
Decifragem ocorreu com sucesso!!!!
Mensagem decifrada foi: Viva a Estruturas Criptográficas :)


### Algumas experiências

In [47]:
import sys

im = int.from_bytes(b'Nelson Faria', byteorder=sys.byteorder)  # => 30147523307896390225895384398
bm = bin(int.from_bytes(b'Nelson Faria', byteorder=sys.byteorder))  # => '0b11000010110100101110010011000010100011000100000011011100110111101110011011011000110010101001110'
im1 = int(bm, 2)
int(im1).to_bytes(12, byteorder=sys.byteorder)   # precisa do tamanho da string

b'Nelson Faria'

In [ ]:
import numpy as np

v1 = [1,1,0,1,1]
v2 = [[1,1,1,1,0],
     [0,1,1,1,1],
     [1,0,1,1,1],
     [1,1,0,1,1],
     [1,1,1,0,1]]
a = np.array(v1)
b = np.array(v2)
#print(np.multiply(a,b))


# Cria um vetor so com os indices onde temos 1's
def indices1(v):
    
    result = []
    for i in range(len(v)):
        if v[i] == 1:
            result = result + [i]
    return result



# componentwise product of vectors
def componentwise(v1, v2):
    a = np.array(v1)
    b = np.array(v2)
    return np.multiply(a,b)

# componentwise product of index vectors
def componentwiseInd(v1, v2):
    
    result = []
    for i in range(len(v1)):
        for j in range(len(v2)):
            if v1[i] == v2[j]:
                result = result + [v1[i]]
    return result


# BitFlip otimizado
# Recebe como parametro a matriz H, o sindroma s, o numero de 1's que o vetor h0 e h1 possui, 
# pelo que cada vetor possui w/2 bits iguais a 1(correspondera ao numero de linhas da matriz H) 
# e ainda recebe o numero de colunas da matriz H (é o r)
def bitFlip(H, s, w, r):
    
    e = [0]*r
    s1 = np.array(s)
    Ht = np.transpose(H)
    # Enquanto o numero de 1´s do sindroma nao for 0
    while len(s1) != 0:
        
        j = 0
        # Percorrer todas as linhas da matriz transposta(ou seja, as colunas da matriz H)
        for j in range(r):
            
            # Um dos criterios possiveis para o threshold (Numero de 1´s presentes numa coluna da matriz H 
            # + numero de 1´s do sindroma)
            threshold = len(componentwiseInd(Ht[j], s)) - 5
            # Se o numero de 1's em comum com o sindroma atual for maior que o threshold
            if len(componentwiseInd(Ht[j], s1)) >= threshold:
                
                # Atualiza o erro
                e[j] = mod(e[j] + 1, 2)
                # Atualiza o sindroma atual
                for i in range(len(s1)):
                    for k in range(w):
                        if s1[i] == Ht[j][k]:
                            np.delete(s1,i)
                            break
    
    return e


v2 = [[1,2,3,4,5],
     [1,2,3,4,5],
     [1,2,3,4,5],
     [1,2,3,4,5],
     [1,2,3,4,5]]

# Funcao que gera a matriz de rotacao a partir de um vetor de indices e ainda do numero de colunas que este deve ter
def rot2(v, tam):
    
    result = np.array([[0]*tam]*len(v))
    # Serve para gerar a matriz de rotação
    for i in range(len(v)):
        count = 0
        for j in range(tam):
            result[i, j] = mod(v[i] + count, tam)
            count += 1
    return result

# Concatenar 2 matrizes 
def matrix_union(A, B):
    for a, b in zip(A, B):
        yield [*a, *b]
    

v = [1,2,3,4,5]
v1 = [6,7,8,9,10]
print(componentwiseInd(v, v1))
# H = (rot(h0), rot(h1))
H = list(matrix_union(rot2(v, 8), rot2(v1, 10)))
print(H)
#print(v1 != np.array([0]*len(v1)))
print(bitFlip(H, v, 5, 8))